In [ ]:
%tensorflow_version 1.x
import tensorflow as tf

TensorFlow 1.x selected.


In [ ]:
# install and import nltk package 
!sudo pip install -U nltk
import nltk
nltk.download("punkt")

In [ ]:
from gensim.models import Word2Vec
import pandas as pd
import keras
import numpy as np
import sklearn as sk

Using TensorFlow backend.


In [ ]:
data = pd.read_csv("Ethos_Dataset_Binary.csv",delimiter=";")
label = data["isHate"]
comments = data["comment"]
target = label.to_numpy()
for i in range(len(target)):
  if target[i] >= .5:
    target[i] = 1
  else:
    target[i] = 0

Preprocessing dataset 

In [ ]:
from nltk import word_tokenize
import string
table = str.maketrans('', '', string.punctuation)

for i in range(len(comments)):
  comments[i] = word_tokenize(comments[i])
  comments[i]=[w.lower() for w in comments[i]]
  stripped = [w.translate(table) for w in comments[i]]
  comments[i] = [w for w in stripped if w.isalpha()]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
def dict(x):
    dic = []
    counts = []
    for item in x:
        for word in item:
            if word in dic:
                index = dic.index(word)
                counts[index] += 1
            else:
                dic.append(word)
                counts.append(1)
    return dic, counts

dic, counts = dict(comments)
wordcount = len(dic)
window_size = 10
rarewords_threshold = 3

build co-occurrence matrix 

In [ ]:
cooc_matr = np.zeros([wordcount, wordcount])
for item in comments:
    for i in range(len(item)):
        cenword = item[i]
        cen_id = dic.index(cenword)
        for j in range(i - window_size, i + window_size + 1):
            if j in range(len(item)):
                val = window_size + 1 - abs(i - j)
                coword = item[j]
                co_id = dic.index(coword)
                cooc_matr[cen_id][co_id] += val

for i in range(wordcount):
    cooc_matr[i] = cooc_matr[i] / cooc_matr[i][i] * 1.0

Getting rarewords

In [ ]:
rarewords = []
for i in range(wordcount):
    if counts[i] < rarewords_threshold:
        rarewords.append(i)

In [ ]:
#Local sensitivity hashing

Use local sensitivity hashing on co-occurrence matrix 

In [ ]:
pip install lshashpy3

In [ ]:
from lshashpy3 import LSHash
hashsize = 200
friends = 5
#lsh = LSHash(window_size, wordcount)
lsh = LSHash(hashsize, wordcount)
for i in range(len(cooc_matr)):
    lsh.index(cooc_matr[i])

friendslist = []
sim_matr = []
for i in range(len(rarewords)):
    result = lsh.query(cooc_matr[rarewords[i]], num_results=friends, distance_func='euclidean')
    length = len(result)
    fri = []
    sim = []
    for j in range(length):
        fri.append(str(np.array(result[j][0])))
        sim.append(float(result[j][1]))
    if length < friends:
        count = friends - length
        for i in range(count):
            fri.append(str(8))
            sim.append(-1)
    friendslist.append(fri)
    sim_matr.append(sim)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  del sys.path[0]


Start trainning a embedding to approximate LSH 

In [ ]:
dim = 200
learning_rate = 0.01
training_epoch = 50
X = tf.placeholder("float", [wordcount, wordcount])
sim_true = tf.placeholder("float", [len(rarewords), friends])
sim_pred = tf.placeholder("float", [len(rarewords), friends])
n_hidden_1 = 200
n_hidden_2 = dim

weights = {
    'encoder_h1': tf.Variable(tf.random_normal([wordcount, n_hidden_1])),
    'encoder_h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'decoder_h1': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_1])),
    'decoder_h2': tf.Variable(tf.random_normal([n_hidden_1, wordcount])),
}
biases = {
    'encoder_b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'encoder_b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'decoder_b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'decoder_b2': tf.Variable(tf.random_normal([wordcount])),
}

def encoder(x):
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['encoder_h1']),
                                   biases['encoder_b1']))
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['encoder_h2']),
                                   biases['encoder_b2']))
    return layer_2

def decoder(x):
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['decoder_h1']),
                                   biases['decoder_b1']))
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['decoder_h2']),
                                   biases['decoder_b2']))
    return layer_2

cooc_true = X
encoder_op = encoder(X)
cooc_pred = decoder(encoder_op)
cost = tf.reduce_mean(tf.pow(cooc_true - cooc_pred, 2))
cost_min = cost + tf.reduce_mean(tf.pow(sim_true - sim_pred, 2))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost_min)

if int((tf.__version__).split('.')[1]) < 12 and int((tf.__version__).split('.')[0]) < 1:
    init = tf.initialize_all_variables()
else:
    init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.graph.finalize()
    sess.run(init)
    for epoch in range(training_epoch):
        word_list = sess.run(encoder_op, feed_dict={X: cooc_matr})

        lsh = LSHash(hashsize, dim)
        for item in word_list:
            lsh.index(item)

        cos = []
        for i in range(len(rarewords)):
            result = lsh.query(word_list[rarewords[i]], num_results=friends, distance_func='euclidean')
            cosin = -1 * np.ones(friends)
            for item in result:
                id = str(np.array(item[0]))
                for j in range(friends):
                    if id == friendslist[i][j]:
                        cosin[j] = float(item[1])
            cos.append(cosin)

        cost_fin, _ = sess.run([cost_min, optimizer], feed_dict={sim_pred: cos, sim_true: sim_matr, X: cooc_matr})
        print("epoch:",epoch, ",cost=", "{:.9f}".format(cost_fin))
    print("Optimization Finished!")
    encoder_result = sess.run(encoder_op, feed_dict={X: cooc_matr})

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:60: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


epoch: 0 ,cost= 0.659681857
epoch: 1 ,cost= 0.610038936
epoch: 2 ,cost= 0.566093504
epoch: 3 ,cost= 0.528135777
epoch: 4 ,cost= 0.494091809
epoch: 5 ,cost= 0.463117778
epoch: 6 ,cost= 0.434807777
epoch: 7 ,cost= 0.409109950
epoch: 8 ,cost= 0.386078835
epoch: 9 ,cost= 0.366051644
epoch: 10 ,cost= 0.349022150
epoch: 11 ,cost= 0.334097505
epoch: 12 ,cost= 0.320876658
epoch: 13 ,cost= 0.309738696
epoch: 14 ,cost= 0.300465912
epoch: 15 ,cost= 0.292656451
epoch: 16 ,cost= 0.286193073
epoch: 17 ,cost= 0.280950785
epoch: 18 ,cost= 0.276715040
epoch: 19 ,cost= 0.273261905
epoch: 20 ,cost= 0.270221919
epoch: 21 ,cost= 0.267396957
epoch: 22 ,cost= 0.264886707
epoch: 23 ,cost= 0.262795031
epoch: 24 ,cost= 0.261137486
epoch: 25 ,cost= 0.259684950
epoch: 26 ,cost= 0.258180976
epoch: 27 ,cost= 0.256685942
epoch: 28 ,cost= 0.255419850
epoch: 29 ,cost= 0.254364043
epoch: 30 ,cost= 0.253394842
epoch: 31 ,cost= 0.252517700
epoch: 32 ,cost= 0.251750797
epoch: 33 ,cost= 0.251137078
epoch: 34 ,cost= 0.25067

load Word2Vec 

In [ ]:
W2v = Word2Vec(comments)

Begin word similarity task, using wordsim355 

In [ ]:
from scipy import spatial
wordsim= pd.read_csv("wordsim.txt",delimiter = "\t")
word1 = wordsim["tiger"]
word2 = wordsim["cat"]
similarity = wordsim["7.35"]
word11 = []
word22 = []
w2v1 = []
w2v2 = []
simw = []
sim33 = []
sim99 = []

In [ ]:
for i in range(len(word1)):
  if word1[i] in dic and word2[i] in dic:
    word1Embedding = encoder_result[dic.index(word1[i])]
    word2Embedding = encoder_result[dic.index(word2[i])]
    sim = 1 - spatial.distance.cosine(word1Embedding,word2Embedding)
    sim = np.multiply(sim,10)
    w2vsim = "Missing Word"
    if word1[i] in W2v and word2[i] in W2v:
      w2vsim = 1 - spatial.distance.cosine(W2v[word1[i]],W2v[word2[i]])
      w2vsim = np.multiply(w2vsim,10)
    word11.append(word1[i])
    word22.append(word2[i])
    sim33.append(sim)    
    sim99.append(similarity[i])
    simw.append(w2vsim)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':


In [ ]:
for i in range(len(word11)):
  print("word1",word11[i]," word2",word22[i]," LSHWE Similarity",sim33[i]," word2vec",simw[i] ," wordsim Similarity", sim99[i])

word1 tiger  word2 tiger  LSHWE Similarity 10.0  word2vec Missing Word  wordsim Similarity 10.0
word1 smart  word2 stupid  LSHWE Similarity 9.8575758934021  word2vec Missing Word  wordsim Similarity 5.81
word1 king  word2 queen  LSHWE Similarity 9.729621410369873  word2vec Missing Word  wordsim Similarity 8.58
word1 fuck  word2 sex  LSHWE Similarity 9.931595921516418  word2vec 9.996511340141296  wordsim Similarity 9.44
word1 drink  word2 eat  LSHWE Similarity 9.825449585914612  word2vec Missing Word  wordsim Similarity 6.87
word1 money  word2 dollar  LSHWE Similarity 9.713850021362305  word2vec Missing Word  wordsim Similarity 8.42
word1 tiger  word2 animal  LSHWE Similarity 9.653976559638977  word2vec Missing Word  wordsim Similarity 7.0
word1 planet  word2 sun  LSHWE Similarity 9.836339950561523  word2vec Missing Word  wordsim Similarity 8.02
word1 skin  word2 eye  LSHWE Similarity 9.720719456672668  word2vec Missing Word  wordsim Similarity 6.22
word1 life  word2 death  LSHWE Simila

Get embedding

In [ ]:
embedding_matrix = {}
for i in range(len(dic)):
  embedding_matrix[dic[i]] = encoder_result[i]


Add special token - empty and unknown

In [ ]:
Unk = "UNK"
uv = np.zeros(25)
embedding_matrix[Unk] = uv
dic.append(Unk)
empty = "EMP"
ev = np.zeros(25)
embedding_matrix[empty] = ev
dic.append(empty)
print(len(embedding_matrix))
print(type(embedding_matrix))

3654
<class 'dict'>


Process embedding informaiton to get sum and average vectors for each sentence. 

In [ ]:
maxl = 12
def TTI(sentence):
  result = []
  for i in sentence:
    if i in dic:
      result.append(dic.index(i))
    else:
      result.append(dic.index("UNK"))
  while len(result) < 12:
    result.append(dic.index("EMP"))
  return result[:12]

In [ ]:
def lsh_avg(sentence):
  result = np.zeros(200)
  for i in sentence:
    if i in dic:
      result += encoder_result[dic.index(i)]
  return result/len(sentence)


In [ ]:
def lsh_sum(sentence):
  result = np.zeros(200)
  for i in sentence:
    if i in dic:
      result += encoder_result[dic.index(i)]
  return result

In [ ]:
def w2v_avg(sentence):
  result = np.zeros(100)
  for i in sentence:
    if i in W2v:
      result += W2v[i]
  return result/len(sentence)

In [ ]:
def w2v_sum(sentence):
  result = np.zeros(100)
  for i in sentence:
    if i in W2v:
      result += W2v[i]
  return result

In [ ]:
Lav = []
Lsu = []
Wav = []
Wsu = []
for i in range(len(comments)):
    s = comments[i]
    Lav.append(lsh_avg(s))
    Lsu.append(lsh_sum(s))
    Wav.append(w2v_avg(s))
    Wsu.append(w2v_sum(s))


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


In [ ]:
Lav, Lsu, Wav,Wsu,la = sk.utils.shuffle(Lav, Lsu, Wav,Wsu,target,random_state=42)

In [ ]:
with open('lshwe_avg.npy', 'wb') as f:
  np.save(f,Lav)
with open('lshwe_sum.npy', 'wb') as f:
  np.save(f,Lsu)
with open('w2v_avg.npy', 'wb') as f:
  np.save(f,Wav)
with open('w2v_sum.npy', 'wb') as f:
  np.save(f,Wsu)
with open('label.npy', 'wb') as f:
  np.save(f,la)